In [146]:
import pandas as pd
import numpy as np
import re

In [147]:
#load and inspect data
df = pd.read_csv('properties.csv')
df.head()

,Price,Property_Type,Bed,Bath,Parking,Location,Link
0,"$2,975 per week",Apartment / Unit / Flat,3 Beds,2 Baths,2 Parking,SYDNEY NSW 2000,https://www.domain.com.au/1301-61-macquarie-st...
1,"$1,250.00 per week furnished (6 Month Lease On...",Apartment / Unit / Flat,2 Beds,2 Baths,− Parking,SYDNEY NSW 2000,https://www.domain.com.au/807-50-murray-street...
2,"$2,975 per week",Apartment / Unit / Flat,3 Beds,2 Baths,2 Parking,SYDNEY NSW 2000,https://www.domain.com.au/1301-61-macquarie-st...
3,$850 per week,Apartment / Unit / Flat,1 Bed,1 Bath,− Parking,SYDNEY NSW 2000,https://www.domain.com.au/129-harrington-stree...
4,Furnished | $800 per week,Apartment / Unit / Flat,1 Bed,1 Bath,1 Parking,SYDNEY NSW 2000,https://www.domain.com.au/206-132-sussex-st-sy...


In [148]:
def filter_property_types(df):
    # Select desired property types
    desired_property_types = ['Apartment / Unit / Flat',  'House', 'Studio', 'Terrace', 'Townhouse']
    df = df[df['Property_Type'].isin(desired_property_types)]
    return df

In [149]:
def get_number(row,column):
    number = 0  #set number=0 (return 0 if no number present)
    for item in row[column]:
        if item.isdigit():
            number = int(item)
    return number       

In [150]:
def get_feature_count(df,features=['Bed','Bath','Parking']):
    for feature in features:
        #df[feature] = df.apply(lambda row: get_number(row,feature), axis=1)
        df.loc[:, feature] = df.apply(lambda row: get_number(row, feature), axis=1)
    return df
    

In [151]:
def get_price(row):
    price_string = row['Price']
    match = re.search(r'\$([\d,]+)', price_string) #return price in numbers including comma
    if match:
        #get first price captured, remove comma to leave only numbers and convert to int
        amount = match.group(1).replace(',', '')
        return int(amount)
    else:
        return None

In [152]:
def clean_data(df):

    df = get_feature_count(df)
    
    #extract postcode as last item of 'Location' string
    df['Postcode'] = df['Location'].apply(
        lambda location: location.split()[-1])

    #create new column 'Suburb' from 'Location'
    df['Suburb'] = df['Location'].apply(
        lambda location:' '.join(location.split()[:-2]))

    #get price as integer
    df['priceInt'] = df.apply(lambda row: get_price(row), axis=1)
    df = df.dropna() #drop properties where no price is included

    #remove properties with no bathroom (suspect rooms in sharehouse)
    df = df[df['Bath'] != 0]

    #remove properties priced below $250 per week as suspect
    df = df[df['priceInt'] >= 250]

    #check if price is listed per month
    prices_to_exclude = ['/month', 'per month',]

    #filter out properties with prices listed as per month
    df = df[~df['Price'].str.contains('|'.join(prices_to_exclude), case=False)]

    df = filter_property_types(df)
    df.reset_index(drop=True, inplace=True)
    
    return df   
    

In [153]:
cleaned_df = clean_data(df)
cleaned_df.head()

,Price,Property_Type,Bed,Bath,Parking,Location,Link,Postcode,Suburb,priceInt
0,"$2,975 per week",Apartment / Unit / Flat,3,2,2,SYDNEY NSW 2000,https://www.domain.com.au/1301-61-macquarie-st...,2000,SYDNEY,2975.0
1,"$1,250.00 per week furnished (6 Month Lease On...",Apartment / Unit / Flat,2,2,0,SYDNEY NSW 2000,https://www.domain.com.au/807-50-murray-street...,2000,SYDNEY,1250.0
2,"$2,975 per week",Apartment / Unit / Flat,3,2,2,SYDNEY NSW 2000,https://www.domain.com.au/1301-61-macquarie-st...,2000,SYDNEY,2975.0
3,$850 per week,Apartment / Unit / Flat,1,1,0,SYDNEY NSW 2000,https://www.domain.com.au/129-harrington-stree...,2000,SYDNEY,850.0
4,Furnished | $800 per week,Apartment / Unit / Flat,1,1,1,SYDNEY NSW 2000,https://www.domain.com.au/206-132-sussex-st-sy...,2000,SYDNEY,800.0


In [154]:
#cleaned_df.to_csv('sydney_cleaned_df.csv')